In [78]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

test = "../data/raw/test.csv"
train = "../data/raw/train.csv"
sample_submission = "../data/raw/sample_submission.csv"

test = pd.read_csv(test)
train = pd.read_csv(train)
sample_submission = pd.read_csv(sample_submission)

### Understanding the Data

Predictors:
- brand
- model
- model_year
- milage
- fuel_type
- engine
- transmission
- ext_color
- int_col
- accident
- clean_title

Target:
- price

In [84]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [31]:
list(train.select_dtypes("number"))

['id', 'model_year', 'milage', 'price']

In [22]:
train.select_dtypes("number").describe()

,id,model_year,milage,price
count,188533.000000,188533.000000,188533.000000,1.885330e+05
mean,94266.000000,2015.829998,65705.295174,4.387802e+04
std,54424.933488,5.660967,49798.158076,7.881952e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47133.000000,2013.000000,24115.000000,1.700000e+04
50%,94266.000000,2017.000000,57785.000000,3.082500e+04
75%,141399.000000,2020.000000,95400.000000,4.990000e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [21]:
train.select_dtypes("number").isna().sum()

id            0
model_year    0
milage        0
price         0
dtype: int64

In [23]:
train.select_dtypes("object").isna().sum()

brand               0
model               0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
dtype: int64

In [82]:
train.ext_col.nunique()

16

### Data Cleaning

Predictors:
- brand
- model
- model_year
- milage
- fuel_type
- engine
- transmission
- ext_color
- int_col
- accident
- clean_title

In [77]:
train.brand.value_counts().head()

brand
Ford             23088
Mercedes-Benz    19172
BMW              17028
Chevrolet        16335
Audi             10887
Name: count, dtype: int64

In [78]:
# Applying feature engineering - one hot encoding
train = pd.get_dummies(train, columns=['brand'])

In [79]:
train.columns

Index(['id', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price', 'brand_Acura', 'brand_Alfa', 'brand_Aston', 'brand_Audi',
       'brand_BMW', 'brand_Bentley', 'brand_Bugatti', 'brand_Buick',
       'brand_Cadillac', 'brand_Chevrolet', 'brand_Chrysler', 'brand_Dodge',
       'brand_FIAT', 'brand_Ferrari', 'brand_Ford', 'brand_GMC',
       'brand_Genesis', 'brand_Honda', 'brand_Hummer', 'brand_Hyundai',
       'brand_INFINITI', 'brand_Jaguar', 'brand_Jeep', 'brand_Karma',
       'brand_Kia', 'brand_Lamborghini', 'brand_Land', 'brand_Lexus',
       'brand_Lincoln', 'brand_Lotus', 'brand_Lucid', 'brand_MINI',
       'brand_Maserati', 'brand_Maybach', 'brand_Mazda', 'brand_McLaren',
       'brand_Mercedes-Benz', 'brand_Mercury', 'brand_Mitsubishi',
       'brand_Nissan', 'brand_Plymouth', 'brand_Polestar', 'brand_Pontiac',
       'brand_Porsche', 'brand_RAM', 'brand_Rivian', 'brand_Rolls-Royce',
   

In [ ]:
def update_ext_col(df):
    df.loc[(df['ext_col'].str.contains('Black', case=False, na=False)), 'ext_col'] = 'Black'
    df.loc[(df['ext_col'].str.contains('Noir', case=False, na=False)), 'ext_col'] = 'Black'
    df.loc[(df['ext_col'].str.contains('Blue', case=False, na=False)), 'ext_col'] = 'Blue'
    df.loc[(df['ext_col'].str.contains('Blu', case=False, na=False)), 'ext_col'] = 'Blue'
    df.loc[(df['ext_col'].str.contains('Red', case=False, na=False)), 'ext_col'] = 'Red'
    df.loc[(df['ext_col'].str.contains('White', case=False, na=False)), 'ext_col'] = 'White'
    df.loc[(df['ext_col'].str.contains('Green', case=False, na=False)), 'ext_col'] = 'Green'
    df.loc[(df['ext_col'].str.contains('Gray', case=False, na=False)), 'ext_col'] = 'Gray'
    df.loc[(df['ext_col'].str.contains('Grey', case=False, na=False)), 'ext_col'] = 'Gray'
    df.loc[(df['ext_col'].str.contains('Silver', case=False, na=False)), 'ext_col'] = 'Silver'
    df.loc[(df['ext_col'].str.contains('Metallic', case=False, na=False)), 'ext_col'] = 'Metallic'
    df.loc[(df['ext_col'].str.contains('Yellow', case=False, na=False)), 'ext_col'] = 'Yellow'
    df.loc[(df['ext_col'].str.contains('Orange', case=False, na=False)), 'ext_col'] = 'Orange'
    colors = ['Black', 'Blue', 'Red','White','Green','Gray','Silver','Metallic','Gold','Brown','Orange','Beige','Yellow','Purple','Pink']

    # Use .str.contains() to create a mask for rows containing any word from words_list
    # We use '|'.join to create a regular expression that matches any word in words_list
    pattern = '|'.join(colors)
    mask = df['ext_col'].str.contains(pattern, case=False, regex=True)

    # Change the value if the text does NOT contain any word from the list
    df.loc[~mask, 'ext_col'] = 'Others'
    return df

def update_int_col(df):
    df.loc[(df['ext_col'].str.contains('Black', case=False, na=False)), 'ext_col'] = 'Black'
    df.loc[(df['ext_col'].str.contains('Noir', case=False, na=False)), 'ext_col'] = 'Black'
    df.loc[(df['ext_col'].str.contains('Blue', case=False, na=False)), 'ext_col'] = 'Blue'
    df.loc[(df['ext_col'].str.contains('Blu', case=False, na=False)), 'ext_col'] = 'Blue'
    df.loc[(df['ext_col'].str.contains('Red', case=False, na=False)), 'ext_col'] = 'Red'
    df.loc[(df['ext_col'].str.contains('White', case=False, na=False)), 'ext_col'] = 'White'
    df.loc[(df['ext_col'].str.contains('Green', case=False, na=False)), 'ext_col'] = 'Green'
    df.loc[(df['ext_col'].str.contains('Gray', case=False, na=False)), 'ext_col'] = 'Gray'
    df.loc[(df['ext_col'].str.contains('Grey', case=False, na=False)), 'ext_col'] = 'Gray'
    df.loc[(df['ext_col'].str.contains('Silver', case=False, na=False)), 'ext_col'] = 'Silver'
    df.loc[(df['ext_col'].str.contains('Metallic', case=False, na=False)), 'ext_col'] = 'Metallic'
    df.loc[(df['ext_col'].str.contains('Yellow', case=False, na=False)), 'ext_col'] = 'Yellow'
    df.loc[(df['ext_col'].str.contains('Orange', case=False, na=False)), 'ext_col'] = 'Orange'
    colors = ['Black', 'Blue', 'Red','White','Green','Gray','Silver','Metallic','Gold','Brown','Orange','Beige','Yellow','Purple','Pink']

    # Use .str.contains() to create a mask for rows containing any word from words_list
    # We use '|'.join to create a regular expression that matches any word in words_list
    pattern = '|'.join(colors)
    mask = df['ext_col'].str.contains(pattern, case=False, regex=True)

    # Change the value if the text does NOT contain any word from the list
    df.loc[~mask, 'ext_col'] = 'Others'
    return df

In [ ]:
train = pd.get_dummies(train, columns=['brand','fuel_type',''])

In [50]:
train.nunique()

id              188533
brand               57
model             1897
model_year          34
milage            6651
fuel_type            9
engine            1117
transmission        52
ext_col            319
int_col            156
accident             2
clean_title          1
price             1569
dtype: int64

In [47]:
train.accident.value_counts()

accident
None reported                             144514
At least 1 accident or damage reported     41567
Name: count, dtype: int64

### Train Test Split

In [26]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [82]:
x_train = train[['model_year', 'milage','brand_Ford','brand_Mercedes-Benz','brand_BMW','brand_Chevrolet','brand_Audi']]
y_train = train["price"]

x_test = test[['model_year','milage']]

### Regression

Linear Regression Model

In [83]:
lin_reg = LinearRegression()

In [84]:
lin_reg.fit(x_train, y_train)

LinearRegression()

### Evaluation

In [85]:
pred = lin_reg.predict(x_train)

print("MAE", mean_absolute_error(pred, y_train))
print("RMSE", mean_squared_error(pred, y_train, squared=False))
print("R2 score", lin_reg.score(x_train, y_train))

MAE 24031.477956752253
RMSE 75431.81416101585
R2 score 0.08410895503862437


In [87]:
lin_reg_coef = {feature : coef for feature, coef in zip(x_train.columns, lin_reg.coef_)}
lin_reg_coef

{'model_year': 1089.4082081105862,
 'milage': -0.36514513109448604,
 'brand_Ford': -3321.429421655943,
 'brand_Mercedes-Benz': 1644.915045887571,
 'brand_BMW': -3148.769489461429,
 'brand_Chevrolet': -1500.9421277693991,
 'brand_Audi': -7407.41394184485}